Chargement des données nettoyées
Identification de TOUS les sous-genres
Création des colonnes is_contemporary, is_historical, etc.
Statistiques complètes par sous-genre
Sauvegarde finale avec sous-genres